# *Task 2 (Analytical).*

_Ви - аналітик в стартапі, вам потрібно допомогти
маркетинговій команді розрахувати ключові показники. Ваш стартап - додаток
у якого моделлю монетизації є підписка. Спочатку юзер бере тріал (він триває 7
днів), після закінчення тріалу він може відписатись, або ж продовжити платити за
тижневу підписку. Підписка коштує 9.99 за тиждень. Юзер може відписатись у
будь-який момент, після цього він не буде більше платити._

Ваше завдання - розрахувати LTV за допомогою Python. Будемо оцінювати
ваше загальне розуміння завдання та результат. У файлі data_analytics.csv ви
знайдете фрагмент бази даних, що вам потрібна, де:

* Event Date – дата коли відбулась подія (тріал чи продовження підписки).

* Subscription Offer Type – тип підписки (якщо вказано Free Trial - значить це
початок тріалу, якщо поле пусте - це підписка).

* LTV – lifetime value. У цьому випадку рахується через конверсії з тріалу в
оплату і потім в наступні оплати. 

$$LTV = ([1]+[2]+[3]+[4]+[5]),$$ де [1] = (dev proceeds * Conversion to trial), а [2] = [1] * 1st
purchase to 2nd, [3] = [2] * 2nd to 3rd і тд.

__Важливо!__ Apple Store забирає собі 30% від вартості підписки. LTV в цьому
випадку ми будемо рахувати на 6 тижнів(1 тиждень тріалу та 5 підписки).
Розв язок першого завдання потрібно завантажити за посиланням.


## Рішення аналітика стартапу:

### 1. Підключаємо необхідні модулі (ну бо без того буде трішки важче)

In [1]:
import pandas as pd
import numpy as np

### 2. Читаємо дані з файлу (цей крок суперважливий, бо без нього LTV саме рахуватися не захоче)

також поглянемо на те, що ми прочитали з файлу, щоб переглянути, чи всі дані є важливими

In [2]:
data = pd.read_csv("data_analytics.csv")
data.head()

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
0,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,IDR,IDR,iPhone,ID,223411322456765433,NaN,NaN,NaN,1
1,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,USD,USD,iPhone,US,223411322456765434,NaN,NaN,NaN,1
2,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,INR,INR,iPhone,IN,223411322456765435,NaN,NaN,NaN,1
3,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,GBP,GBP,iPhone,GB,223411322456765437,NaN,NaN,NaN,1
4,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,INR,INR,iPhone,IN,223411322456765438,NaN,NaN,NaN,1


In [3]:
data.tail()

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
2375,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766566,NaN,NaN,NaN,1
2376,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,USD,USD,iPhone,US,223411322456766569,NaN,NaN,NaN,1
2377,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,INR,INR,iPhone,IN,223411322456766571,NaN,NaN,NaN,1
2378,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,USD,USD,iPhone,US,223411322456766572,NaN,NaN,NaN,1
2379,2019-08-24,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,NaN,NaN,NaN,MYR,MYR,iPhone,MY,223411322456766573,NaN,NaN,NaN,1


### 3. Шукаємо необхідні стовпчики для реалізації алгоритму, зазначеного в умові

Дослідивши вміст таблиці та умову завдання, доречним буде відкинути зайві стовпчики, які не містять потрібних нам даних (App Apple ID,	Subscription Name,	Standard Subscription Duration,	Promotional Offer Name,	Promotional Offer ID,	Marketing Opt-In Duration,	Customer Currency,	Proceeds Currency, Device,	Country, Refund,	Purchase Date,	Units), ну бо і в умові немає нічого про те, чи вони нам хоч чимось допоможуть, та й без них не буде стільки "води"

Єдине зі вищезазначеного списку, що можна було б залишити - це Proceeds Currency, адже у таблиці міститься 34 унікальні валюти (знайдено за допомогою *len(data["Proceeds Currency"].unique())*), але за умовою в нас є умовні одиниці замість валют, тому цю частину упустимо


In [4]:
data = data.drop(['App Apple ID', 'Subscription Name', 'Standard Subscription Duration',
                'Promotional Offer Name', 'Promotional Offer ID', 'Marketing Opt-In Duration',
                'Customer Currency', 'Proceeds Currency', 'Device', 'Country', 'Refund', 'Purchase Date', 'Units'], axis=1)
data.head()

,Event Date,Subscription Offer Type,Subscription Offer Duration,Subscriber ID,Subscriber ID Reset
0,2019-07-15,Free Trial,7 Days,223411322456765433,NaN
1,2019-07-15,Free Trial,7 Days,223411322456765434,NaN
2,2019-07-15,Free Trial,7 Days,223411322456765435,NaN
3,2019-07-15,Free Trial,7 Days,223411322456765437,NaN
4,2019-07-15,Free Trial,7 Days,223411322456765438,NaN


### 4. Підраховуємо необхідні для обчислень початкові дані

Далі необхідно визначити скільки взагалі є рядків з підпискою (2380 побачили, як вивели хвіст таблиці, але переконатись потрібно) та дізнатися, скільки взагалі було унікальних користувачів та рядків з інформацією про платні підписки (бо всі і так починають з тріалу)

+можна дізнатись, скільки загалом у нас днів у таблиці, щоб дізнатись період, протягом якого ми слідкуємо за підписками

In [5]:
CountOfRows = len(data["Subscriber ID"])
CountOfUniqueSubscribers = len(data["Subscriber ID"].unique())
CountOfPaidSubscriptions = CountOfRows - CountOfUniqueSubscribers ##те саме, що виведення усіх рядків, де Subscription Offer Type не є "Free Trial"
Dates = pd.to_datetime(data["Event Date"])
print(f"""
Кількість рядків таблиці: {CountOfRows}
Кількість користувачів за увесь час(себто з {Dates.min()} по {Dates.max()}): {CountOfUniqueSubscribers}
Кількість платних тижневих підписок: {CountOfPaidSubscriptions}
Кількість днів у таблиці в період з {Dates.min()} по {Dates.max()}: {(Dates.max() - Dates.min()).days}""")


Кількість рядків таблиці: 2380
Кількість користувачів за увесь час(себто з 2019-07-15 00:00:00 по 2019-08-24 00:00:00): 1019
Кількість платних тижневих підписок: 1361
Кількість днів у таблиці в період з 2019-07-15 00:00:00 по 2019-08-24 00:00:00: 40


З отриманих даних можна зрозуміти, що спостерігали ми 40 днів, тобто 5 тижнів і 5 днів і звідси можна заокруглити до 6 тижнів (бо платить користувач за кожен тиждень підписки (оскільки не вказано, чи плата стягувалась на початку, чи в кінці тижня, то будемо рахувати, що проплачено у нас уже 6 тижнів))

Також можна побачити, що кількість платних підписок за усі 5 тижнів є меншою, ніж кількість користувачів, які поласували тріалом (в ідеалі мало б бути 1019 * 5 = 5095, але в нас лише 1361... тож підрахуємо скільки дат (і відповідно скільки тижнів) той чи інший користувач юзав нашу апку:

In [6]:
data.groupby('Subscriber ID').count()

,Event Date,Subscription Offer Type,Subscription Offer Duration,Subscriber ID Reset
Subscriber ID,,,,
223411322456765433,6,1,1,0
223411322456765434,6,1,1,0
223411322456765435,6,1,1,0
223411322456765437,6,1,1,0
223411322456765438,6,1,1,0
...,...,...,...,...
223411322456766800,1,1,1,0
223411322456766801,1,1,1,0
223411322456766802,1,1,1,0


1019 -тобто порахували для кожного унікального користувача період використання ним апки. А тепер бажано підрахувати ще й за кількістю дат, скільки користувачів юзали апку 1го, ..., 6го тижня 

In [7]:
CountOfSubscribersEachWeek = []

for i in range(6):
    CountOfSubscribersEachWeek.append((data.groupby('Subscriber ID').count()['Event Date'] > i).sum())
print(CountOfSubscribersEachWeek)

[1019, 499, 353, 262, 155, 92]


### 5. Розраховуємо LTV

Розрахунок нашого LTV (ну не нашого, в принципі, але розрахунок кожного користувача буде таким):

1. Якщо користувач насолодився тріалом і відписався, то ми не отримуємо нічого, нічого і не винні і в результаті виходимо в 0
2. Якщо більше тижнів, то виконуємо наступні дії:
    *розраховуємо Dev proceeds — скільки отримує розробник після вирахування комісій.
    *розраховуємо Conversion from trial — конверсія з тріалу в оплату і потім в наступні оплати (на основі вищеотриманих даних про користувачів).
    *беручи формулу з умови, де $$LTV = ([1]+[2]+[3]+[4]+[5]),$$ де [1] = (dev proceeds * Conversion to trial), а [2] = [1] * 1st purchase to 2nd, [3] = [2] * 2nd to 3rd і тд, за допомогою циклу реалізовуємо її.

In [8]:
def LTV(NumberOfWeeksWhenUsed):
    #1:
    if NumberOfWeeksWhenUsed == 1:
        return 0
    #2.1:
    DevProceeds= 9.99 * (1 - 0.3)
    #2.2:
    ConversionToTrial =  CountOfSubscribersEachWeek[1] / CountOfSubscribersEachWeek[0]
    #2.3:
    LifetimeValue = [DevProceeds * ConversionToTrial]
    for i in range(2, NumberOfWeeksWhenUsed):
                                                    #2.2:
        LifetimeValue.append(LifetimeValue[-1] * (CountOfSubscribersEachWeek[i] / CountOfSubscribersEachWeek[i - 1]))
    return sum(LifetimeValue)
        
print(f"""Результати:
LTV від користувача, що юзав апку протягом безкоштовного тижня (тріалу): {LTV(1)};
LTV від користувача, що юзав апку протягом одного тижня:                 {LTV(2)};
LTV від користувача, що юзав апку протягом двох тижнів:                  {LTV(3)};
LTV від користувача, що юзав апку протягом трьох тижнів:                 {LTV(4)};
LTV від користувача, що юзав апку протягом чотирьох тижнів:              {LTV(5)};
LTV від користувача, що юзав апку протягом п'яти тижня:                  {LTV(6)}.""")

Результати:
LTV від користувача, що юзав апку протягом безкоштовного тижня (тріалу): 0;
LTV від користувача, що юзав апку протягом одного тижня:                 3.4244425907752696;
LTV від користувача, що юзав апку протягом двох тижнів:                  5.8469440628066724;
LTV від користувача, що юзав апку протягом трьох тижнів:                 7.644947988223748;
LTV від користувача, що юзав апку протягом чотирьох тижнів:              8.708652600588813;
LTV від користувача, що юзав апку протягом п'яти тижня:                  9.340012757605496.


### 6. Створюємо результуючу таблицю з LTV для кожного користувача

Також можемо додати результуючу таблицю, яка міститиме LTV кожного користувача і виведемо унікальні значення LTV, щоб, про всякий, звірити дані з нашою функцією)))

In [9]:
results = data.groupby('Subscriber ID').count()
results['Lifetime Value'] = results['Event Date'].apply(LTV)
print(results["Lifetime Value"].unique())

[9.34001276 8.7086526  7.64494799 5.84694406 3.42444259 0.        ]


### 7. Про всяк випадок, перевіримо нашу функцію, розрахувавши LTV "вручну"

Для безкоштовного тижня LTV = 0

Далі робимо розрахунок за формулою, що була дана в умові:

(нагадаємо, скільки користувачів "дожили" до останнього тижня, щоб не гортати файлик: [1019, 499, 353, 262, 155, 92])

In [10]:
#[1] = (dev proceeds * Conversion to trial) = 3.42444259077527 
one = 9.99*(1 - 0.3) * 499/1019 
#[2] = [1] * 1st purchase to 2nd = 2.422501472031403 
two = one * 353/499 
#[3] = 1.798003925417076
three = two * 262/353 
#[4] = 1.063704612365064 
four = three * 155/262 
#[5] = 0.631360157016683
five = four * 92/155 

print(f"""Отже: (LTV вручну vs LTV через функцію)
LTV від користувача, що юзав апку протягом безкоштовного тижня (тріалу): 0                 vs  {LTV(1)};
LTV від користувача, що юзав апку протягом одного тижня:                 {one}  vs  {LTV(2)};
LTV від користувача, що юзав апку протягом двох тижнів:                  {one + two} vs  {LTV(3)}; 
LTV від користувача, що юзав апку протягом трьох тижнів:                 {one + two + three} vs  {LTV(4)};
LTV від користувача, що юзав апку протягом чотирьох тижнів:              {one + two + three + four} vs  {LTV(5)};
LTV від користувача, що юзав апку протягом п'яти тижня:                  {one + two + three + four + five} vs  {LTV(6)};
""")

Отже: (LTV вручну vs LTV через функцію)
LTV від користувача, що юзав апку протягом безкоштовного тижня (тріалу): 0                 vs  0;
LTV від користувача, що юзав апку протягом одного тижня:                 3.4244425907752696  vs  3.4244425907752696;
LTV від користувача, що юзав апку протягом двох тижнів:                  5.8469440628066724 vs  5.8469440628066724; 
LTV від користувача, що юзав апку протягом трьох тижнів:                 7.644947988223747 vs  7.644947988223748;
LTV від користувача, що юзав апку протягом чотирьох тижнів:              8.708652600588811 vs  8.708652600588813;
LTV від користувача, що юзав апку протягом п'яти тижня:                  9.340012757605495 vs  9.340012757605496;



Результати співпали, тому можна стверджувати, що функція працює відповідно до очікувань аналітика стартапу
## Дякую за увагу!